In [ ]:
#!pip install duckdb
import ipywidgets as widgets
from datetime import datetime
from IPython.display import Markdown as md
import pandas as pd
import duckdb 
import plotly.express as px
import plotly.graph_objects as go


db = duckdb.connect()

In [ ]:
def load_csv(table_list):
      for table in table_list:
          db.execute(
              f"""
          CREATE TABLE {table["name"]} AS SELECT * FROM '{table["path"]}';
          """
          )

tables= [
    {"name": "praise_table", "path": "data/praises.csv"},
    {"name": "quantification_table", "path": "data/quantifications.csv"},
    {"name": "useraccounts_table", "path": "data/useraccounts.csv"},
    {"name": "period_table", "path": "data/periods.csv"},
    {"name": "users_table", "path": "data/users.csv"},

]

load_csv(tables)

# Praise Analysis and Visualization Dashboard
This document fetches the results of the praise reward system and allows analysis of the resulting reward distribution. 

Please select the dates you want to load into the system. They will be stored in a dataframe called 'praise_df' for further analysis.

In [ ]:
periods_df = db.execute("SELECT * FROM period_table").df()
roundname_list = periods_df.name.unique().tolist()

round = widgets.Dropdown(
    options=roundname_list,
    description='Round:',
    disabled=False,
)

display(round)

In [ ]:
row = periods_df.index[periods_df.name == round.value]

if (row[0] == 0) :
    start_date = datetime.strptime("2021-06-30T23:59:59.999Z", '%Y-%m-%dT%H:%M:%S.%fZ')
else :
    #start_date = datetime.strptime(periods_df.iloc[row-1]['endDate'].values[0], '%Y-%m-%dT%H:%M:%S.%fZ')
    start_date = pd.to_datetime(periods_df.iloc[row-1]['endDate'].values[0])

#end_date = datetime.strptime(periods_df.iloc[row]['endDate'].values[0], '%Y-%m-%dT%H:%M:%S.%fZ')
end_date = pd.to_datetime(periods_df.iloc[row]['endDate'].values[0])

start_date_picker = widgets.DatePicker(
    value= start_date,
    description='Pick a Date',
    disabled=False
)
end_date_picker = widgets.DatePicker(
    value= end_date,
    description='Pick a Date',
    disabled=False
)


In [ ]:
display(start_date_picker)

In [ ]:
display(end_date_picker)

In [ ]:
#start_date_value= periods_df.iloc[row-1]['endDate'].values[0]
#end_date_value = periods_df.iloc[row]['endDate'].values[0]

start_date_value= start_date_picker.value.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
end_date_value = end_date_picker.value.strftime('%Y-%m-%dT%H:%M:%S.%fZ')

# get a DataFrame of the praise data in the specified period
praise_df = db.execute("""
SELECT pt.*, ua_giver.name AS giver_name, ua_receiver.name AS receiver_name
FROM praise_table  pt
JOIN useraccounts_table ua_giver ON pt.giver = ua_giver._id
JOIN useraccounts_table ua_receiver ON pt.receiver = ua_receiver._id
WHERE pt.createdAt BETWEEN ? AND ?
""", [start_date_value,end_date_value]).df()

#clean up the usernames
praise_df['giver_name'] = praise_df['giver_name'].str.split('#').str[0]
praise_df['receiver_name'] = praise_df['receiver_name'].str.split('#').str[0]


## General Information

In [ ]:
md(f"<ul><li>This period covers praise given between <b>{start_date.strftime('%Y-%m-%d %H:%M:%S')}</b> and  <b>{end_date.strftime('%Y-%m-%d %H:%M:%S')}</b>. </li>  \
<li>There were <b>{len(praise_df.index)}</b> instances of praise given during this period. </li>  \
</ul>")


### Rating distribution
Since praise gets valued on a scale, we can take a look at how often quantifiers award each value.
Note: This metric disregards scores of praise marked as a duplicate, since the score of the original is already being taken into account.

In [ ]:
hist = db.execute("""
    SELECT q.score, COUNT(*) as frequency
    FROM quantification_table q 
    INNER JOIN praise_df p ON q.praise = p._id
    GROUP BY q.score
""").df()

fig_freq = px.bar(hist, x="score", y="frequency", labels={"score": "Rating","frequency": "Number of appearances"}, title="Praise Rating Distribution", width=800, height=300)
fig_freq.update_xaxes(type='category')
 


fig_freq.show()

### Top 10 highest rated contributions
The ten highest rated contributions for this round were the following:

In [ ]:
toppraise = db.execute("""
SELECT score, giver_name, receiver_name, reason
FROM praise_df p
ORDER BY p.score DESC
LIMIT 10;
""").df()
toppraise.style.hide(axis="index")

### Praise Giving Distribution
We can take a look at the amount of praise different users gave. The bar represents the amount of praise given by a specfic user, while the line shows the sum of the scores of that praise. 


In [ ]:
giving_dist = db.execute("""
SELECT p.giver_name, COUNT(*) AS amount_given, AVG(p.score) AS avg_score
FROM praise_df p
GROUP BY p.giver_name
ORDER BY amount_given DESC
""").df()

fig_giving = go.Figure()

# Add the bar trace from fig_a
fig_giving.add_trace(go.Bar(
    x=giving_dist["giver_name"],
    y=giving_dist['amount_given'],
    name="Praise Given",
    marker_color='rgba(58, 71, 80, 0.6)',
    yaxis='y1'
))

# Add the line trace from fig_b
fig_giving.add_trace(go.Scatter(
    x=giving_dist["giver_name"],
    y=giving_dist['avg_score'],
    name="Average Score",
    mode='lines+markers',
    line=dict(color='red'),
    yaxis='y2'
))

# Configure the layout
fig_giving.update_layout(
    title="Praise Givers Sorted by Total Score and Average Score",
    xaxis_title="Giver",
    yaxis_title="Praise Given",
    yaxis2=dict(
        title="Average Score",
        overlaying='y',
        side='right'
    ),
    legend_title="",
    hovermode="x unified"
)

# Show the combined plot
fig_giving.show()

### Praise Receiver Distribution
We can also take a look at the amount of praise different users received. The bar represents the amount of praise instances received by a specfic user, while the line shows the average score of the received praise.

In [ ]:
receiving_dist = db.execute("""
SELECT p.receiver_name, COUNT(*) AS amount_received, AVG(p.score) AS avg_score
FROM praise_df p
GROUP BY p.receiver_name
ORDER BY amount_received DESC
""").df()
px.bar(receiving_dist, x="receiver_name",y='amount_received', labels={"receiver_name": "Receiver","amount_received": "Praise Received"},title='Praise Receivers Sorted by Total Score')

# Assuming 'giving_dist' is a pandas DataFrame with columns: 'giver_name', 'amount_given', 'avg_score'
# You might need to modify the code below to match your data format.

# Create a combined Figure object
fig_receiving = go.Figure()

# Add the bar trace from fig_a
fig_receiving.add_trace(go.Bar(
    x=receiving_dist["receiver_name"],
    y=receiving_dist['amount_received'],
    name="Praise Received",
    marker_color='rgba(58, 71, 80, 0.6)',
    yaxis='y1'
))

# Add the line trace from fig_b
fig_receiving.add_trace(go.Scatter(
    x=receiving_dist["receiver_name"],
    y=receiving_dist['avg_score'],
    name="Average Score",
    mode='lines+markers',
    line=dict(color='red'),
    yaxis='y2'
))

# Configure the layout
fig_receiving.update_layout(
    title="Praise Givers Sorted by Total Score and Average Score",
    xaxis_title="Receiver",
    yaxis_title="Praise Received",
    yaxis2=dict(
        title="Average Score",
        overlaying='y',
        side='right'
    ),
    legend_title="",
    hovermode="x unified"
)

# Show the combined plot
fig_receiving.show()

## Amount of praise over time
Here we can observe the amount of praise dished on a weekly basis

In [ ]:
praise_over_time = db.execute("""
SELECT DATE_TRUNC('week', createdAt) AS week_start,
       COUNT(*) AS row_count
FROM praise_df
GROUP BY week_start
ORDER BY week_start;
""").df()
fig_praise_over_time = px.line(praise_over_time, x="week_start",y='row_count', labels={"week_start": "Weeks","row_count": "Amount of Praise"},  title='Praise Given Over Time')
fig_praise_over_time.show()

## Givers and receivers over time
Here we can observe the amount of unique 
users who either dished or received praise on any given week.

In [ ]:
participants_over_time = db.execute("""
SELECT DATE_TRUNC('week', createdAt) AS week_start_date,
       COUNT(DISTINCT giver) AS unique_givers,
       COUNT(DISTINCT receiver) AS unique_receivers
FROM praise_df
GROUP BY week_start_date
ORDER BY week_start_date;
""").df()
fig_participants_over_time = px.line(participants_over_time, x="week_start_date",y=['unique_givers','unique_receivers'], labels={"week_start_date": "Weeks","value": "Amount of Participants", "unique_givers":"Unique Givers", "unique_receivers":"Unique Receivers"}, title='Participants Over Time')
fig_participants_over_time.show()

### Praise Flow

Now for something more fun: let's surface the top "praise flows" from the data.
On one side we have the top 15 praise givers separately, on the other the top 20 receivers. The people outside the selection get aggregated into the "REST FROM" and "REST TO" categories. 

In [ ]:
def prepare_praise_flow(dataframe_in, n_senders, n_receivers):
    # Select required columns and drop rows with missing values
    reference_df = dataframe_in[['FROM', 'TO', 'TOTAL_SCORE']].dropna().reset_index(drop=True)

    # Select top n_senders senders by combined TOTAL_SCORE and rename the rest as 'Rest from 1'
    sender_scores = reference_df.groupby('FROM')['TOTAL_SCORE'].sum()
    top_senders = sender_scores.nlargest(n_senders).index
    reference_df.loc[~reference_df['FROM'].isin(top_senders), 'FROM'] = 'REST FROM'

    # Select top n_receivers receivers by combined TOTAL_SCORE and rename the rest as 'Rest to 1'
    receiver_scores = reference_df.groupby('TO')['TOTAL_SCORE'].sum()
    top_receivers = receiver_scores.nlargest(n_receivers).index
    reference_df.loc[~reference_df['TO'].isin(top_receivers), 'TO'] = 'REST TO'

    # Group by sender and receiver, and sum up the scores
    df4 = reference_df.groupby(['FROM', 'TO']).sum().reset_index()

    # Append a space to the 'TO' column
    df4['TO'] = df4['TO'] + ' '

    return df4



NUMBER_OF_SENDERS_FLOW = 15 #The left side, the praise senders. X largest ones + one bucket for the rest 
NUMBER_OF_RECEIVERS_FLOW = 20 #The right side, the praise receivers. X largest ones + one bucket for the rest 
#dist_for_praise_flow = praise_distribution.rename(columns = {'FROM USER ACCOUNT':'FROM', 'TO USER ACCOUNT':'TO'})

praise_flow = db.execute("""
SELECT giver_name as 'FROM' , receiver_name as 'TO', SUM(score) as 'TOTAL_SCORE'
FROM praise_df
GROUP BY giver_name, receiver_name
""").df()

praise_flow = prepare_praise_flow(praise_flow.copy(), n_senders=NUMBER_OF_SENDERS_FLOW, n_receivers=NUMBER_OF_RECEIVERS_FLOW)



In [ ]:
# Get unique labels for nodes (both 'FROM' and 'TO')
labels = list(set(praise_flow['FROM']).union(set(praise_flow['TO'])))

# Get index of labels
source_indices = [labels.index(src) for src in praise_flow['FROM']]
target_indices = [labels.index(trg) for trg in praise_flow['TO']]

# Create a color mapping for the 'FROM' values
from_labels = praise_flow['FROM'].unique()
colors = px.colors.qualitative.Safe
color_mapping = {label: colors[i % len(colors)] for i, label in enumerate(from_labels)}

# Assign colors to nodes based on 'FROM' values
node_colors = ['rgba(100, 100, 100, 0.5)'] * len(labels)
for idx, label in enumerate(labels):
    if label in color_mapping:
        node_colors[idx] = color_mapping[label]

# Create Sankey diagram
fig_praise_flow = go.Figure(data=[go.Sankey(
    node=dict(
        pad=0,
        thickness=40,
        line=dict(color="black", width=0.5),
        label=labels,
        color=node_colors
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=praise_flow['TOTAL_SCORE'],
        color=[color_mapping[src] for src in praise_flow['FROM']]
    )
)])

# Configure layout
fig_praise_flow.update_layout(
    title_text="Praise flow for Batch 1. Sum of Praise. Left - praise sender. Right - praise receiver",
    font_size=10,
    width=1000,
    height=800,
    plot_bgcolor='snow',
    margin=dict(l=30, r=30, t=50, b=30)
)

# Show the plot
fig_praise_flow.show()


## New Givers
Let's see the amount of people who praise for the first time on a weekly basis.

In [ ]:
new_givers = db.execute("""
SELECT DATE_TRUNC('week', sub.createdAt) as week_start,
       COUNT(DISTINCT sub.giver) as num_new_givers
FROM (SELECT giver, MIN(createdAt) as createdAt
      FROM praise_df
      GROUP BY giver) as sub
WHERE sub.createdAt >= ?
GROUP BY week_start
ORDER BY week_start ASC
""", [start_date_value]).df()

fig_new_givers = px.line(new_givers, x="week_start",y='num_new_givers', labels={"week_start": "Weeks","num_new_givers": "New Givers"}, title='New Givers Over Time')
fig_new_givers.show()